# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources in the README.md file
- Happy learning!

In [206]:
# Import your libraries:

%matplotlib inline

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)

In this lab, we will explore a dataset that describes websites with different features and labels them either benign or malicious . We will use supervised learning algorithms to figure out what feature patterns malicious websites are likely to have and use our model to predict malicious websites.

# Challenge 1 - Explore The Dataset

Let's start by exploring the dataset. First load the data file:

In [207]:
websites = pd.read_csv('../website.csv')

#### Explore the data from a bird's-eye view.

You should already been very familiar with the procedures now so we won't provide the instructions step by step. Reflect on what you did in the previous labs and explore the dataset.

Things you'll be looking for:

* What the dataset looks like?
* What are the data types?
* Which columns contain the features of the websites?
* Which column contains the feature we will predict? What is the code standing for benign vs malicious websites?
* Do we need to transform any of the columns from categorical to ordinal values? If so what are these columns?

Feel free to add additional cells for your explorations. Make sure to comment what you find out.

In [208]:
# Your code here

print(websites.shape)
print("································")
display(websites.head())
print("································")
print(websites.dtypes)
print("································")
websites["Type"].unique()

(1781, 21)
································


,URL,URL_LENGTH,NUMBER_SPECIAL_CHARACTERS,CHARSET,SERVER,CONTENT_LENGTH,WHOIS_COUNTRY,WHOIS_STATEPRO,WHOIS_REGDATE,WHOIS_UPDATED_DATE,TCP_CONVERSATION_EXCHANGE,DIST_REMOTE_TCP_PORT,REMOTE_IPS,APP_BYTES,SOURCE_APP_PACKETS,REMOTE_APP_PACKETS,SOURCE_APP_BYTES,REMOTE_APP_BYTES,APP_PACKETS,DNS_QUERY_TIMES,Type
0,M0_109,16,7,iso-8859-1,nginx,263.0,None,None,10/10/2015 18:21,None,7,0,2,700,9,10,1153,832,9,2.0,1
1,B0_2314,16,6,UTF-8,Apache/2.4.10,15087.0,None,None,None,None,17,7,4,1230,17,19,1265,1230,17,0.0,0
2,B0_911,16,6,us-ascii,Microsoft-HTTPAPI/2.0,324.0,None,None,None,None,0,0,0,0,0,0,0,0,0,0.0,0
3,B0_113,17,6,ISO-8859-1,nginx,162.0,US,AK,7/10/1997 4:00,12/09/2013 0:45,31,22,3,3812,39,37,18784,4380,39,8.0,0
4,B0_403,17,6,UTF-8,None,124140.0,US,TX,12/05/1996 0:00,11/04/2017 0:00,57,2,5,4278,61,62,129889,4586,61,4.0,0


································
URL                           object
URL_LENGTH                     int64
NUMBER_SPECIAL_CHARACTERS      int64
CHARSET                       object
SERVER                        object
CONTENT_LENGTH               float64
WHOIS_COUNTRY                 object
WHOIS_STATEPRO                object
WHOIS_REGDATE                 object
WHOIS_UPDATED_DATE            object
TCP_CONVERSATION_EXCHANGE      int64
DIST_REMOTE_TCP_PORT           int64
REMOTE_IPS                     int64
APP_BYTES                      int64
SOURCE_APP_PACKETS             int64
REMOTE_APP_PACKETS             int64
SOURCE_APP_BYTES               int64
REMOTE_APP_BYTES               int64
APP_PACKETS                    int64
DNS_QUERY_TIMES              float64
Type                           int64
dtype: object
································


array([1, 0])

Para mi modelo voy a utilizar la variable Type como groundtruth, y voy a usar el resto como features para predecir si la web es maligna (1) o benigna (1).

Aquí he comprobado si 1 es maligno o benigno: 
https://www.kaggle.com/shikhar14cse/learning-model-using-sklearn/data

In [209]:
# Miro a ver dónde tengo valores nulos.
null_cols = websites.isnull().sum()
null_cols[null_cols > 0]

SERVER               1
CONTENT_LENGTH     812
DNS_QUERY_TIMES      1
dtype: int64

In [210]:
# Casi la mitad de los valores de mi columna CONTENT_LENGTH es nulo. Por tanto, no me puede
# aportar mucho, así que me deshago de ella.

websites.drop("CONTENT_LENGTH", axis = 1, inplace = True)

In [211]:
# Hay varias variables cuyos valores no son numéricos. Voy a ojearlas para decidir si me 
# sirven (y si las convierto a dummies o cambio sus valores por números) o no (y me deshago de ellas).
# También tendré que convertir a ordinales las fechas (WHOIS_REGDATE y WHOIS_UPDATED_DATE).

In [212]:
# Analizo la columna URL

print(len(websites["URL"].unique()))

# Demasiados valores categóricos distintos. Me deshago de ella.

websites.drop("URL", axis = 1, inplace = True)

1781


In [213]:
# Analizo la columna CHARSET

print(websites["CHARSET"].unique())

['iso-8859-1' 'UTF-8' 'us-ascii' 'ISO-8859-1' 'utf-8' 'None'
 'windows-1251' 'ISO-8859' 'windows-1252']


In [214]:
websites["CHARSET"].value_counts()

UTF-8           676
ISO-8859-1      427
utf-8           379
us-ascii        155
iso-8859-1      134
None              7
ISO-8859          1
windows-1251      1
windows-1252      1
Name: CHARSET, dtype: int64

In [215]:
print(len(websites["CHARSET"]))

1781


In [216]:
# Veo que hay valores que quieren decir lo mismo pero que están escritos de forma distinta. Lo corrijo.

# Creo un diccionario con los valores y sus correspondientes correcciones.

charsets = {"UTF-8": "UTF-8", "ISO-8859-1": "ISO-8859-1", "utf-8": "UTF-8", 
            "iso-8859-1": "ISO-8859-1", "ISO-8859": "ISO-8859-1", 
            "windows-1251": "windows", "windows-1252": "windows"}

In [217]:
# Defino una función para corregir los valores.

def unify_charsets(c):
    return charsets.get(c)

In [218]:
# Aplico la función a toda la columna.

websites["CHARSET"] = websites["CHARSET"].apply(unify_charsets)

In [219]:
websites["CHARSET"].value_counts()

UTF-8         1055
ISO-8859-1     562
windows          2
Name: CHARSET, dtype: int64

In [220]:
# Todos los valores que eran "None" en CHARSET van a ser nulos ahora. No voy a querer nulos
# así que me deshago de ellos

null_cols = websites.isnull().sum()
null_cols[null_cols > 0]

CHARSET            162
SERVER               1
DNS_QUERY_TIMES      1
dtype: int64

In [221]:
websites = websites.dropna()

In [222]:
# Ahora puedo hacer dummies con la columna CHARSET.

for cs in websites["CHARSET"].unique():
    websites[cs] = websites["CHARSET"].str.contains(cs, regex = False).astype(int)
    
websites.drop("CHARSET", axis = 1, inplace = True)
websites.head()

,URL_LENGTH,NUMBER_SPECIAL_CHARACTERS,SERVER,WHOIS_COUNTRY,WHOIS_STATEPRO,WHOIS_REGDATE,WHOIS_UPDATED_DATE,TCP_CONVERSATION_EXCHANGE,DIST_REMOTE_TCP_PORT,REMOTE_IPS,APP_BYTES,SOURCE_APP_PACKETS,REMOTE_APP_PACKETS,SOURCE_APP_BYTES,REMOTE_APP_BYTES,APP_PACKETS,DNS_QUERY_TIMES,Type,ISO-8859-1,UTF-8,windows
0,16,7,nginx,None,None,10/10/2015 18:21,None,7,0,2,700,9,10,1153,832,9,2.0,1,1,0,0
1,16,6,Apache/2.4.10,None,None,None,None,17,7,4,1230,17,19,1265,1230,17,0.0,0,0,1,0
3,17,6,nginx,US,AK,7/10/1997 4:00,12/09/2013 0:45,31,22,3,3812,39,37,18784,4380,39,8.0,0,1,0,0
4,17,6,None,US,TX,12/05/1996 0:00,11/04/2017 0:00,57,2,5,4278,61,62,129889,4586,61,4.0,0,0,1,0
5,18,7,nginx,SC,Mahe,3/08/2016 14:30,3/10/2016 3:45,11,6,9,894,11,13,838,894,11,0.0,0,0,1,0


In [223]:
# Analizo la columna SERVER

print(websites["SERVER"].unique()[:5])

['nginx' 'Apache/2.4.10' 'None' 'Apache/2' 'nginx/1.10.1']


In [224]:
# Demasiados valores categóricos complejos para meter en mi modelo. Me deshago de la columna.

websites.drop("SERVER", axis = 1, inplace = True)

In [225]:
# Analizo la columna WHOIS_COUNTRY

print(websites["WHOIS_COUNTRY"].unique())
print(websites["WHOIS_COUNTRY"].value_counts()[:10])

['None' 'US' 'SC' 'GB' 'UK' 'RU' 'AU' 'CA' 'PA' 'IN' 'LU' 'TH'
 "[u'GB'; u'UK']" 'FR' 'NL' 'JP' 'CN' 'SE' 'SI' 'IL' 'ru' 'KY' 'AT' 'CZ'
 'PH' 'BE' 'NO' 'TR' 'LV' 'DE' 'ES' 'BR' 'us' 'HK' 'KR' 'UA' 'CH' 'BS'
 'PK' 'IT' 'Cyprus' 'BY' 'AE' 'IE' 'UY' 'KG']
US      1000
None     272
CA        77
ES        63
AU        29
PA        21
GB        17
JP        11
UK        10
IN         9
Name: WHOIS_COUNTRY, dtype: int64


In [226]:
# Puedo dividir los países entre si son registros de USA (1) o no (0)

def country_tonum(c):
    if c == "USA":
        return 1
    else:
        return 0
    
websites["WHOIS_COUNTRY"] = websites["WHOIS_COUNTRY"].apply(country_tonum)

In [227]:
# Analizo la columna WHOIS_STATEPRO

print(websites["WHOIS_STATEPRO"].unique()[:10])

['None' 'AK' 'TX' 'Mahe' 'CO' 'Kansas' 'Novosibirskaya obl.' 'CA'
 'Tennessee' 'Vi']


In [228]:
# Demasiados valores categóricos complejos para mi modelo. Me deshago de la columna.

websites.drop("WHOIS_STATEPRO", axis = 1, inplace = True)

In [229]:
websites.head(10)

,URL_LENGTH,NUMBER_SPECIAL_CHARACTERS,WHOIS_COUNTRY,WHOIS_REGDATE,WHOIS_UPDATED_DATE,TCP_CONVERSATION_EXCHANGE,DIST_REMOTE_TCP_PORT,REMOTE_IPS,APP_BYTES,SOURCE_APP_PACKETS,REMOTE_APP_PACKETS,SOURCE_APP_BYTES,REMOTE_APP_BYTES,APP_PACKETS,DNS_QUERY_TIMES,Type,ISO-8859-1,UTF-8,windows
0,16,7,0,10/10/2015 18:21,None,7,0,2,700,9,10,1153,832,9,2.0,1,1,0,0
1,16,6,0,None,None,17,7,4,1230,17,19,1265,1230,17,0.0,0,0,1,0
3,17,6,0,7/10/1997 4:00,12/09/2013 0:45,31,22,3,3812,39,37,18784,4380,39,8.0,0,1,0,0
4,17,6,0,12/05/1996 0:00,11/04/2017 0:00,57,2,5,4278,61,62,129889,4586,61,4.0,0,0,1,0
5,18,7,0,3/08/2016 14:30,3/10/2016 3:45,11,6,9,894,11,13,838,894,11,0.0,0,0,1,0
6,18,6,0,29/07/2002 0:00,1/07/2016 0:00,12,0,3,1189,14,13,8559,1327,14,2.0,0,1,0,0
8,20,5,0,8/11/2014 7:41,None,0,0,0,0,2,3,213,146,2,2.0,1,0,1,0
9,20,5,0,8/11/2014 7:41,None,0,0,0,0,2,1,62,146,2,2.0,1,0,1,0
10,20,6,0,14/09/2007 0:00,9/09/2015 0:00,0,0,0,0,0,0,0,0,0,0.0,0,0,1,0
12,20,7,0,11/10/2002 0:00,6/10/2016 0:00,16,6,8,1492,20,20,2334,1784,20,4.0,0,0,1,0


In [230]:
websites[websites["WHOIS_REGDATE"] == '2002-03-20T23:59:59.0Z']
type(websites["WHOIS_REGDATE"][104])

str

In [231]:
# Tengo que convertir los valores de las fechas a ordinales.

from datetime import datetime as dt

def date_toord(date):
    if date == "None":
        return None
    elif date == "2002-03-20T23:59:59.0Z":
        return dt.strptime(str("20/03/2002 23:59"), "%d/%m/%Y %H:%M").date().toordinal()
    else:
        a = dt.strptime(str(date), "%d/%m/%Y %H:%M").date()
        return a.toordinal()

# websites["WHOIS_REGDATE"] = websites["WHOIS_REGDATE"].apply(date_toord)

In [232]:
# WHOIS_REGDATE me está dando demasiados problemas, porque tiene muchos valores sucios. 
# Por tanto me voy a deshacer de ella.

websites.drop("WHOIS_REGDATE", axis = 1, inplace = True)

In [233]:
# Pruebo con WHOIS_UPDATED_DATE

# websites["WHOIS_UPDATED_DATE"] = websites["WHOIS_UPDATED_DATE"].apply(date_toord)

# Me da problemas también, así que me deshago de ella de la misma forma.

websites.drop("WHOIS_UPDATED_DATE", axis = 1, inplace = True)

In [234]:
websites.dtypes

URL_LENGTH                     int64
NUMBER_SPECIAL_CHARACTERS      int64
WHOIS_COUNTRY                  int64
TCP_CONVERSATION_EXCHANGE      int64
DIST_REMOTE_TCP_PORT           int64
REMOTE_IPS                     int64
APP_BYTES                      int64
SOURCE_APP_PACKETS             int64
REMOTE_APP_PACKETS             int64
SOURCE_APP_BYTES               int64
REMOTE_APP_BYTES               int64
APP_PACKETS                    int64
DNS_QUERY_TIMES              float64
Type                           int64
ISO-8859-1                     int64
UTF-8                          int64
windows                        int64
dtype: object

In [235]:
print(websites.shape)
websites.head()

(1617, 17)


,URL_LENGTH,NUMBER_SPECIAL_CHARACTERS,WHOIS_COUNTRY,TCP_CONVERSATION_EXCHANGE,DIST_REMOTE_TCP_PORT,REMOTE_IPS,APP_BYTES,SOURCE_APP_PACKETS,REMOTE_APP_PACKETS,SOURCE_APP_BYTES,REMOTE_APP_BYTES,APP_PACKETS,DNS_QUERY_TIMES,Type,ISO-8859-1,UTF-8,windows
0,16,7,0,7,0,2,700,9,10,1153,832,9,2.0,1,1,0,0
1,16,6,0,17,7,4,1230,17,19,1265,1230,17,0.0,0,0,1,0
3,17,6,0,31,22,3,3812,39,37,18784,4380,39,8.0,0,1,0,0
4,17,6,0,57,2,5,4278,61,62,129889,4586,61,4.0,0,0,1,0
5,18,7,0,11,6,9,894,11,13,838,894,11,0.0,0,0,1,0


#### Next, evaluate if the columns in this dataset are strongly correlated.

In the Mushroom supervised learning lab we did recently, we mentioned we are concerned if our dataset has strongly correlated columns because if it is the case we need to choose certain ML algorithms instead of others. We need to evaluate this for our dataset now.

Luckily, most of the columns in this dataset are ordinal which makes things a lot easier for us. In the next cells below, evaluate the level of collinearity of the data.

We provide some general directions for you to consult in order to complete this step:

1. You will create a correlation matrix using the numeric columns in the dataset.

1. Create a heatmap using `seaborn` to visualize which columns have high collinearity.

1. Comment on which columns you might need to remove due to high collinearity.

In [ ]:
# Your code here



In [ ]:
# Your comment here

# Challenge 2 - Remove Column Collinearity.

From the heatmap you created, you should have seen at least 3 columns that can be removed due to high collinearity. Remove these columns from the dataset.

Note that you should remove as few columns as you can. You don't have to remove all the columns at once. But instead, try removing one column, then produce the heatmap again to determine if additional columns should be removed. As long as the dataset no longer contains columns that are correlated for over 90%, you can stop. Also, keep in mind when two columns have high collinearity, you only need to remove one of them but not both.

In the cells below, remove as few columns as you can to eliminate the high collinearity in the dataset. Make sure to comment on your way so that the instructional team can learn about your thinking process which allows them to give feedback. At the end, print the heatmap again.

In [ ]:
# Your code here


In [ ]:
# Your comment here

In [ ]:
# Print heatmap again


# Challenge 3 - Handle Missing Values

The next step would be handling missing values. **We start by examining the number of missing values in each column, which you will do in the next cell.**

In [ ]:
# Your code here


If you remember in the previous labs, we drop a column if the column contains a high proportion of missing values. After dropping those problematic columns, we drop the rows with missing values.

#### In the cells below, handle the missing values from the dataset. Remember to comment the rationale of your decisions.

In [ ]:
# Your code here


In [ ]:
# Your comment here

#### Again, examine the number of missing values in each column. 

If all cleaned, proceed. Otherwise, go back and do more cleaning.

In [ ]:
# Examine missing values in each column


# Challenge 4 - Handle `WHOIS_*` Categorical Data

There are several categorical columns we need to handle. These columns are:

* `URL`
* `CHARSET`
* `SERVER`
* `WHOIS_COUNTRY`
* `WHOIS_STATEPRO`
* `WHOIS_REGDATE`
* `WHOIS_UPDATED_DATE`

How to handle string columns is always case by case. Let's start by working on `WHOIS_COUNTRY`. Your steps are:

1. List out the unique values of `WHOIS_COUNTRY`.
1. Consolidate the country values with consistent country codes. For example, the following values refer to the same country and should use consistent country code:
    * `CY` and `Cyprus`
    * `US` and `us`
    * `SE` and `se`
    * `GB`, `United Kingdom`, and `[u'GB'; u'UK']`

#### In the cells below, fix the country values as intructed above.

In [ ]:
# Your code here


Since we have fixed the country values, can we convert this column to ordinal now?

Not yet. If you reflect on the previous labs how we handle categorical columns, you probably remember we ended up dropping a lot of those columns because there are too many unique values. Too many unique values in a column is not desirable in machine learning because it makes prediction inaccurate. But there are workarounds under certain conditions. One of the fixable conditions is:

#### If a limited number of values account for the majority of data, we can retain these top values and re-label all other rare values.

The `WHOIS_COUNTRY` column happens to be this case. You can verify it by print a bar chart of the `value_counts` in the next cell to verify:

In [ ]:
# Your code here


#### After verifying, now let's keep the top 10 values of the column and re-label other columns with `OTHER`.

In [ ]:
# Your code here


Now since `WHOIS_COUNTRY` has been re-labelled, we don't need `WHOIS_STATEPRO` any more because the values of the states or provinces may not be relevant any more. We'll drop this column.

In addition, we will also drop `WHOIS_REGDATE` and `WHOIS_UPDATED_DATE`. These are the registration and update dates of the website domains. Not of our concerns.

#### In the next cell, drop `['WHOIS_STATEPRO', 'WHOIS_REGDATE', 'WHOIS_UPDATED_DATE']`.

In [ ]:
# Your code here


# Challenge 5 - Handle Remaining Categorical Data & Convert to Ordinal

Now print the `dtypes` of the data again. Besides `WHOIS_COUNTRY` which we already fixed, there should be 3 categorical columns left: `URL`, `CHARSET`, and `SERVER`.

In [ ]:
# Your code here


#### `URL` is easy. We'll simply drop it because it has too many unique values that there's no way for us to consolidate.

In [ ]:
# Your code here


#### Print the unique value counts of `CHARSET`. You see there are only a few unique values. So we can keep it as it is.

In [ ]:
# Your code here

`SERVER` is a little more complicated. Print its unique values and think about how you can consolidate those values.

#### Before you think of your own solution, don't read the instructions that come next.

In [ ]:
# Your code here


![Think Hard](../think-hard.jpg)

In [ ]:
# Your comment here


Although there are so many unique values in the `SERVER` column, there are actually only 3 main server types: `Microsoft`, `Apache`, and `nginx`. Just check if each `SERVER` value contains any of those server types and re-label them. For `SERVER` values that don't contain any of those substrings, label with `Other`.

At the end, your `SERVER` column should only contain 4 unique values: `Microsoft`, `Apache`, `nginx`, and `Other`.

In [ ]:
# Your code here


In [ ]:
# Count `SERVER` value counts here


OK, all our categorical data are fixed now. **Let's convert them to ordinal data using Pandas' `get_dummies` function ([documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)).** Make sure you drop the categorical columns by passing `drop_first=True` to `get_dummies` as we don't need them any more. **Also, assign the data with dummy values to a new variable `website_dummy`.**

In [ ]:
# Your code here


Now, inspect `website_dummy` to make sure the data and types are intended - there shouldn't be any categorical columns at this point.

In [ ]:
# Your code here


# Challenge 6 - Modeling, Prediction, and Evaluation

We'll start off this section by splitting the data to train and test. **Name your 4 variables `X_train`, `X_test`, `y_train`, and `y_test`. Select 80% of the data for training and 20% for testing.**

In [ ]:
from sklearn.model_selection import train_test_split

# Your code here:


#### In this lab, we will try two different models and compare our results.

The first model we will use in this lab is logistic regression. We have previously learned about logistic regression as a classification algorithm. In the cell below, load `LogisticRegression` from scikit-learn and initialize the model.

In [1]:
# Your code here:



Next, fit the model to our training data. We have already separated our data into 4 parts. Use those in your model.

In [2]:
# Your code here:



finally, import `confusion_matrix` and `accuracy_score` from `sklearn.metrics` and fit our testing data. Assign the fitted data to `y_pred` and print the confusion matrix as well as the accuracy score

In [3]:
# Your code here:



What are your thoughts on the performance of the model? Write your conclusions below.

In [4]:
# Your conclusions here:



#### Our second algorithm is is K-Nearest Neighbors. 

Though is it not required, we will fit a model using the training data and then test the performance of the model using the testing data. Start by loading `KNeighborsClassifier` from scikit-learn and then initializing and fitting the model. We'll start off with a model where k=3.

In [5]:
# Your code here:



To test your model, compute the predicted values for the testing sample and print the confusion matrix as well as the accuracy score.

In [6]:
# Your code here:



#### We'll create another K-Nearest Neighbors model with k=5. 

Initialize and fit the model below and print the confusion matrix and the accuracy score.

In [7]:
# Your code here:



Did you see an improvement in the confusion matrix when increasing k to 5? Did you see an improvement in the accuracy score? Write your conclusions below.

In [8]:
# Your conclusions here:



# Bonus Challenge - Feature Scaling

Problem-solving in machine learning is iterative. You can improve your model prediction with various techniques (there is a sweetspot for the time you spend and the improvement you receive though). Now you've completed only one iteration of ML analysis. There are more iterations you can conduct to make improvements. In order to be able to do that, you will need deeper knowledge in statistics and master more data analysis techniques. In this bootcamp, we don't have time to achieve that advanced goal. But you will make constant efforts after the bootcamp to eventually get there.

However, now we do want you to learn one of the advanced techniques which is called *feature scaling*. The idea of feature scaling is to standardize/normalize the range of independent variables or features of the data. This can make the outliers more apparent so that you can remove them. This step needs to happen during Challenge 6 after you split the training and test data because you don't want to split the data again which makes it impossible to compare your results with and without feature scaling. For general concepts about feature scaling, click [here](https://en.wikipedia.org/wiki/Feature_scaling). To read deeper, click [here](https://medium.com/greyatom/why-how-and-when-to-scale-your-features-4b30ab09db5e).

In the next cell, attempt to improve your model prediction accuracy by means of feature scaling. A library you can utilize is `sklearn.preprocessing.RobustScaler` ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html)). You'll use the `RobustScaler` to fit and transform your `X_train`, then transform `X_test`. You will use logistic regression to fit and predict your transformed data and obtain the accuracy score in the same way. Compare the accuracy score with your normalized data with the previous accuracy data. Is there an improvement?

In [ ]:
# Your code here